## 3.1 SQL 概述

### 3.1.2 SQL 的特点

1. 综合统一

数据系统的主要功能是通过数据库支持的数据语言来实现的
非关系模型(层次模型、网状模型)的语言一般都分为：
- 模式数据定义语言(Schema Data Definition Language, 模式 DDL)
- 外模式数据定义语言(Subschema Data Definition Language, 外模式 DDL 或子模式 DDL)
- 数据存储有关的描述语言(Data Storage Description Language, DSAL)
- 数据操纵语言(Data Manipulation Language, DML)

它们分别用于定义模式、外模式、内模式和进行数据的存储域处置。

SQL 集数据定义语言、数据操纵语言、数据控制语言功能于一体，语言风格统一，可以独立完成数据库生命周期中的全部活动，包括以下一系列操作要求：
- 定义和修改、删除关系模式，定义和删除视图，插入数据，建立数据库。
- 对数据库中的数据进行查询和更新
- 数据库重构和维护
- 数据安全性、完整性控制，以及事务控制。
- 嵌入式 SQL 和 动态 SQL 定义。

2. 高度非过程化

只要提出“做什么”，而无须指明“怎么做”，因此无须了解存取路径。存取路径的选择以及 SQL 的操作过程由系统自动完成。

3. 面向集合的操作方式

4. 以同一种语法结构提供多种使用方式

5. 语言简洁，易学易用

### 3.1.3 SQL 基本概念

支持 SQL 的关系数据库管理系统同样支持关系数据库三级模式结构。其中外模式包括若干视图(view)和部分基本表(base table)，数据库模式包括若干基本表，内模式包括若干存储文件(stored file)。



## 3.2 学生-课程数据库

包括以下三个表：
- 学生表: Student(Sno(key), Sname, Ssex, Sage, Sdept)
- 课程表: Course(Cno(key), Cname, Cpno, Ccredit)
- 学生选课表: SC(Sno(key), Cno(key), Grade)




## 3.3 数据定义

| 操作对象 |               | 操作方式 |             |
| -------- | ------------- | ----------- | ----------- |
|          | 创建        | 删除      | 修改      |
| 模式   | CREATE SCHEMA | DROP SCHEMA |             |
| 表      | CREATE TABLE  | DROP TABLE  | ALTER TABLE |
| 视图   | CREATE VIEW   | DROP VIEW   |             |
| 索引   | CREATE INDEX  | DROP INDEX  | ALTER INDEX |

SQL 标准不提供修改模式定义和修改视图定义的操作。用户如果想修改这些对象，只能先将它们删除然后再重建。SQL 标准也没用提供索引相关的语句，但为了提高查询效率，数据库管理系统通常都提供了索引机制和相关的语句。

现代的关系数据库管理系统提供了一共层次和的数据库对象命名机制，一共关系数据库管理系统的实例中可以建立多个数据库，一个数据库中可以建立多个模式，一个模式下通常包括多个表、视图和索引等数据库对象。

### 3.3.1 模式的定义与删除

1. 定义模式

    `CREATE SCHEMA<模式名>AUTHORIZATION<用户名>;`
    
如果没有指定<模式名>，那么<模式名>隐含为<用户名>

例：为用户 WANG 定义一个学生-课程模式 S-T  

    `CREATE SCHEMA "S-T" AUTHORIZATION WANG;`

定义模式实际上定义了一共命名空间，在这个空间中可以进一步定义该模式包含的数据库对象，例如基本表、视图、索引等。

目前，在 CREAT SCHEMA 中可以接受 CREATE TABLE、CREATE VIEW 和 GRANT 子句。也就是说用户可以在创建模式的同时在这个模式定义中进一步创建基本表、视图、定义授权。

    `CREATE SCHEMA<模式名>AUTHORIZATION<用户名>; [<表定义子句>]|[<视图定义子句>]|[<授权定义子句>] ;`

例子：
    `CREATE SCHEMA TEST AUTHORIZATION ZHANG`
    `CREATE TABLE TAB1  (COL1 SMALLINT,`
                        `COL2 INT,`
                        `COL3 CHAR(20),`
                        `COL4 NUMERIC(10,3)`
                        `)`


2. 删除模式

    `DROP SCHEMA<模式名><CASCADE|RESTRICT>;`

其中 CASCADE 和 RESTRICT 必选其一。CASCADE(级联)，表示在删除模式的同时把该模式中所有的数据库对象全部删除；RESTRICT(限制)，表示如歌该模式中以及定义了下属数据库对象，则拒绝该删除语句的执行。

例子: `DROP SCHEMA ZHANG CASCADE`

### 3.3.2 基本表的定义、删除与修改

1. 定义基本表

创建了一共模式就建立了一个数据库的命名空间，一个框架。在这个空间中首先要定义的是该模式包含的数据库基本表。

    `CREAT TABLE<表名>  (<列名><数据类型>[列级完整性约束条件]`
                        `[,<列名><数据类型>[列级完整性约束条件]]`
                        `[,<列名><数据类型>[列级完整性约束条件]]`
                        `[,<列名><数据类型>[列级完整性约束条件]]`
                        `[,[表级完整性约束条件]]);`

例：建立一个 “学生” 表 Student
```sql
    CREATE TABLE Student
        (Sno char(9) PRIMARY KEY,   /* 列级完整约束条件，Sno是主码 */
        Sname CHAR(20) UNIQUE,      /* Sname 取唯一值 */
        Ssex CHAR(2),
        Sage SMALLINT,
        Sdept CHAR(20)
        );
```

系统执行改语句后，就在数据库中建立一共新的空学生表 Student。

```sql
    CREATE TABLE Course
        (Cno char(4) PRIMARY KEY,   /* 列级完整约束条件，Cno是主码 */
        Cname CHAR(40) NOT NULL,    /* Cname 不能取空值 */
        Cpno CHAR(4),               /* Cpno 为前置课 */
        Ccredit SMALLINT,
        FOREIGN KEY (Cpno) REFERENCES Course(Cno)
                                    /* 表级完整性约束条件，Cpno是外码，被参照表是 Course，被参照列是 Cno */
        );
```
**参照表和被参照表可以是同一个表**

```sql
    CREATE TABLE SC
        (Sno char(9),   
        Cno char(4),  
        Grade SMALLINT, 
        PRIMARY KEY (Sno, Cno),     /* 主码由两个属性构成，必须作为表级完整性进行定义 */
        FOREIGN KEY (Sno) REFERENCES Student(Sno),
                                    /* 表级完整性约束条件，Sno是外码，被参照表是 Student，被参照列是 Sno */
        FOREIGN KEY (Cno) REFERENCES Course(Cno)
                                    /* 表级完整性约束条件，Cno是外码，被参照表是 Course，被参照列是 Cno */
        );
```

2. 数据模型

关系模型一个很重要的概念是域。每一个属性来自一个域，它的取值必须是域中的值。

在 SQL 中域的概念用数据类型来实现，定义表的各个属性时需要指明其数据类型及长度。SQL 标准支持多种数据类型。下面列出常用数据类型。注意：不同的关系数据库管理系统中支持的数据类型不完全相同

| 数据类型                    | 含义                                         |
| ------------------------------- | ---------------------------------------------- |
| CHAR (n), CHARACTER(n)          | 长度为 n 的定长字符串                 |
| VARCHAR(n), CHARACTERVARYING(n) | 最大长度为 n 的变长字符串           |
| CLOB                            | 字符串大对象                             |
| BLOB                            | 二进制大对象                             |
| INT, INTEGER                    | 长整数(4 字节)                            |
| SMALLINT                        | 短整数(2 字节)                            |
| BIGINT                          | 大整数(8 字节)                            |
| NUMERIC(p, d)                   | 定点数，由 p 位数字组成，小数点后面有 d 位数字 |
| DECIMAL(p, d), DEC(p, d)        | 同上                                         |
| REAL                            | 取决于机器精度的单精度浮点数     |
| DOUBLE PRECISION                | 取决于机器精度的双精度浮点数     |
| FLOAT(n)                        | 可选精度的浮点数，精度至少为 n 位数字 |
| BOOLEAN                         | 逻辑布尔量                                |
| DATE                            | 日期，包含年、月、日，格式为 YYYY-MM-DD |
| TIME                            | 时间，包含一日的时、分、秒，格式为 HH:MM:SS |
| TIMESTAMP                       | 时间戳类型                                |
| INTERVAL                        | 时间间隔类型                             |


3. 模式与表

每一个基本表都属于一个模式，一个模式包含多个基本表。当定义基本表时一般可以有三种方法定义它所属的模式：
- 在表名中显式的给出模式名
  - CREATE TABLE "S-T".Student(...);
- 在创建模式语句中同时创建表
- 设置所属的模式，这样在创建表时表名中不必给出模式名

当用户创建基本表时没有指定模式，系统根据搜索路径来确定该对象所属的模式。

使用下面的语句显示当前的搜索路径

    `SHOW search_pach;`

搜索路径当前的默认值是 `$user, PUBLIC` 其含义是首先搜索与用户名相同的模式名，如果不存在则使用 PUBLIC 模式。

数据库管理员也可以设置搜索路径：
    `SET search_path TO "S-T", PUBLIC`

4. 修改基本表

SQL 语言用 ALTER TABLE 语句修改基本表，一般格式为：
```sql
    ALTER TABLE<表名>
    [ADD [COLUMN] <新列名><数据类型>[完整性约束]]
    [ADD<表级完整性约束>]
    [DROP [COLUMN] <列名> [CASCADE|RESTRICT]]
    [DROP CONSTRAINT <完整性约束名> [CASCADE|RESTRICT]]
    [ALTER COLUMN <列名><数据类型>];
```

例：向 Student 表增加 “入学时间” 列，其数据类型为日期型  
    `ALTER TABLE Student ADD S_entrance DATE;`

例：将年龄的数据类型由字符改为整数  
    `ALTER TABLE Student ALTER COLUMN Sage INT;`

例: 增加课程名称必须取唯一值的约束条件  
    `ALTER TABLE Course ADD UNIQUE(Cname);`

5. 删除基本表

当某个基本表不再需要时，可以使用 DROP TABLE 语句删除它。一般格式为：  
    `DROP TABLE <表名> [CASCADE|RESTRICT];`

例: 删除 Student 表  
    `DROP TABLE Student CASCADE`

### 3.3.3 索引的建立与删除